In [ ]:
function modelStruc = fit_uv_cont(varargin)

avgWin = true;
tolVal = 1e-10;
useMask = logical([1 1 1 1 0]);
usePB = true;
verbose = false;
cycleMax = 500;
modelStruc = [];
visStruc = [];
gainStruc = [];
fitCal = false;
for idx=1:2:numel(varargin)
    tempVal = varargin{idx+1}; %#ok
    eval([varargin{idx},'=tempVal;'])
    clear tempVal
end

useMask = logical(useMask);

if fitCal
    useMask(1:2) = false; %#ok
    usePB = false;
end

if not(isempty(visStruc))
    if isempty(gainStruc)
        gainStruc = visStruc.gains;
    end
    dataVals = visStruc.cross.*gain_mask(gainStruc,visStruc.metaData.baseToAnts,...
        visStruc.time.mjd,visStruc.source.name,gainStruc.gainPrefs)+...
        flag_mask(visStruc.flags,visStruc.metaData);
    dataWeights = 2.*(calc_noise(visStruc)).^(-2);
    uVals = calc_uvlambda(visStruc.baseCoords.u,visStruc.metaData.freq);
    vVals = calc_uvlambda(visStruc.baseCoords.v,visStruc.metaData.freq);
    freqVals = repmat(shiftdim(visStruc.metaData.freq,-2),...
        [visStruc.metaData.nTime visStruc.metaData.nBase 1 1]);

    badData = not(and(isfinite(dataVals.*dataWeights.*uVals.*vVals.*freqVals),...
        ne(dataVals.*dataWeights.*freqVals,0)));
    dataVals(badData) = 0;
    dataWeights(badData) = 0;
    uVals(badData) = 0;
    vVals(badData) = 0;
    freqVals(badData) = 0;

    if avgWin
        dataVals = sum(dataVals.*dataWeights,4)./sum(dataWeights,4);
        uVals = sum(uVals.*dataWeights,4)./sum(dataWeights,4);
        vVals = sum(vVals.*dataWeights,4)./sum(dataWeights,4);
        freqVals = sum(freqVals.*dataWeights,4)./sum(dataWeights,4);
        dataWeights = sum(dataWeights,4);
    end

    if fitCal
        dataVals = sum(dataVals.*dataWeights,2,'omitnan')./sum(dataWeights,2,'omitnan');
        uVals = sum(uVals.*dataWeights,2,'omitnan')./sum(dataWeights,2,'omitnan');
        vVals = sum(vVals.*dataWeights,2,'omitnan')./sum(dataWeights,2,'omitnan');
        freqVals = sum(freqVals.*dataWeights,2,'omitnan')./sum(dataWeights,2,'omitnan');
        dataWeights = sum(dataWeights,2,'omitnan');
    end

    goodData = and(isfinite(dataVals.*dataWeights.*uVals.*vVals.*freqVals),...
        ne(dataVals.*dataWeights.*freqVals,0));
    dataVals = reshape(dataVals(goodData),[],1);
    dataWeights = reshape(dataWeights(goodData),[],1);
    uVals = reshape(uVals(goodData),[],1);
    vVals = reshape(vVals(goodData),[],1);
    freqVals = reshape(freqVals(goodData),[],1);
    cenRA = reshape(nanmedian(visStruc.source.ra).*15,[],1);
    cenDec = reshape(nanmedian(visStruc.source.dec),[],1);
end

if isempty(modelStruc)
    modelStruc = struct();
    modelStruc.fieldName = visStruc.source.name;
    modelStruc.raJ2000 = median(visStruc.source.ra);
    modelStruc.decJ2000 = median(visStruc.source.dec);
    modelStruc.timeRange = [min(real(visStruc.time.mjd)); max(imag(visStruc.time.mjd))];
    modelStruc.freqRange = [min(visStruc.metaData.freq(:)); max(visStruc.metaData.freq(:))];
    modelStruc.fitCoeffs = [real(nansum(dataVals(:).*dataWeights(:))/nansum(dataWeights(:))); 0; mean(visStruc.metaData.freq(:))];
    modelStruc.fitErrs = zeros(3,1);
    modelStruc.souClass = 'c';
    modelStruc.nSou = 1;
end

[offRA, offDec] = calc_coordoffset( ...
    cenRA, cenDec, modelStruc.raJ2000.*15, modelStruc.decJ2000, true,false, true);
offRA = deg2rad(offRA);
offDec = deg2rad(offDec);

rawChiSq = sum((real(dataVals).^2).*dataWeights) + sum((imag(dataVals).^2).*dataWeights);
fitTerms = reshape(cat(1,offRA,offDec,modelStruc.fitCoeffs),[],1);
fitUncert = zeros(size(fitTerms));
if usePB
    [pbPower,pbDistance] = calc_primarybeam(visStruc.antData.antType(1));
else
    pbPower = [];
    pbDistance = [];
end

lastChiSq = nan;
cycleCount = 0;
while le(cycleCount, cycleMax)
    cycleCount = cycleCount + 1;
    [xVals,yVals,weightVals,delVals] = calc_chisq_matrix(fitTerms,dataVals,...
        dataWeights,uVals,vVals,freqVals,useMask,pbDistance,pbPower);
    [fitVals,fitErrs,chiSqVal,nDOF] = fit_linear_model(xVals,yVals,weightVals);
    fitVals = fitVals.*delVals;
    fitErrs = fitErrs.*delVals;
    if lt(chiSqVal, lastChiSq)
        scaleFac = 0.5;
    else
        scaleFac = 1;
    end
    fitTerms(useMask) = fitTerms(useMask) + (fitVals.*scaleFac);
    fitUncert(useMask) = fitErrs;
    if (verbose)
        fprintf('RA: %5.2f Dec: %5.2f Flux: %6.4f +/- %6.4f Sp: %5.3f +/- %5.3f Chi2: %f ScaleFac: %f\n',...
            fitTerms(1)*(3600*180/pi),fitTerms(2)*(3600*180/pi),fitTerms(3),fitUncert(3),fitTerms(4),fitUncert(4),chiSqVal, scaleFac)
    end
    if or(and(le((lastChiSq-chiSqVal),tolVal*lastChiSq),lt(max(abs(fitVals./fitErrs)),1e-2)),...
            or(any(isnan(fitVals)),any(isnan(fitErrs))))
        % Bail
        fprintf('Sou: %15s RA: %5.2f Dec: %5.2f Flux: %6.4f +/- %6.4f Sp: %+5.3f +/- %5.3f Chi2: %f ScaleFac: %f\n',...
            visStruc.source.name, fitTerms(1)*(3600*180/pi),fitTerms(2)*(3600*180/pi),fitTerms(3),fitUncert(3),fitTerms(4),fitUncert(4),chiSqVal, scaleFac)
        break
    end
    lastChiSq = chiSqVal;
end

if le(cycleCount, cycleMax)
    if any(useMask(1:2))
        offRA = rad2deg(fitTerms(1,:));
        offDec = rad2deg(fitTerms(2,:));
        [newRA, newDec] = calc_coordoffset(cenRA, cenDec, offRA, offDec, true,true, true);
        modelStruc.raJ2000 = newRA./15;
        modelStruc.decJ2000 = newDec;
    end
    if any(useMask(3:4))
        modelStruc.fitCoeffs(:) = fitTerms(3:end);
        modelStruc.fitErrs(:) = fitUncert(3:end);
    end

end

end


function [fitVals,fitErrs,chiSqVal,nDOF] = fit_linear_model(xVals,yVals,weightVals)
    alphaMatrix = transpose(xVals)*(xVals.*repmat(weightVals,[1 size(xVals,2)]));
    betaMatrix = transpose(xVals)*(yVals.*weightVals);
    fitVals = alphaMatrix\betaMatrix;
    errVals = yVals-(xVals*fitVals);
    nDOF = numel(yVals)-size(xVals,2);
    chiSqVal = sum((errVals.^2).*weightVals);
    fitErrs = sqrt((chiSqVal/nDOF).*diag(inv(transpose(xVals)*...
        (xVals.*repmat(weightVals,[1 size(xVals,2)])))));
end

function [xVals,yVals,weightVals,delVals] = calc_chisq_matrix(fitTerms,...
   dataVals,dataWeights,uVals,vVals,freqVals,useMask,pbDistance,pbPower)
    % useMask | rxPos decPos fitCoeffs1 fitCoeffs2 fitCoeffs3 ...
    speedC = 29979245800;

    % For right now, this only works w/ point sources!
    % Figure out how big this thing is supposed to be
    nFitParams = sum(useMask(:));
    delVals = zeros(nFitParams,1);
    xVals = zeros(numel(dataVals)*2,sum(useMask));
    [delReal0, delImag0] = calc_ps_cont_response(fitTerms,...
        uVals,vVals,freqVals,pbDistance,pbPower);
    yVals = [(real(dataVals)-delReal0); (imag(dataVals)-delImag0)];
    weightVals = repmat(dataWeights,[2 1]);

    delFlux = 0.1/sqrt(max(dataWeights(:),[],'omitnan'));
    delPos = (0.1/sqrt(max(uVals(:).^2 + vVals(:).^2)))*(speedC/median(freqVals(:)));
    delFreq = max(freqVals(:))/min(freqVals(:));
    useMaskArr = find(useMask);
    for idx=1:nFitParams
        if or(eq(mod(useMaskArr(idx)-1,5),0),eq(mod(useMaskArr(idx)-1,5),1))
            delStep = delPos;
        elseif eq(mod(useMaskArr(idx)-1,5),2)
            delStep = delFlux;
        elseif eq(mod(useMaskArr(idx)-1,5),3)
            delStep = log10((delFlux+abs(fitTerms(useMaskArr(idx)-1)))/...
                abs(fitTerms(useMaskArr(idx)-1)))/log10(delFreq);
        else
            continue
        end
        delVals(idx) = delStep;

        tempFitTerms = fitTerms;
        tempFitTerms(useMaskArr(idx)) = tempFitTerms(useMaskArr(idx)) - (delStep/2);
        [delReal1, delImag1] = calc_ps_cont_response(tempFitTerms,...
            uVals,vVals,freqVals,pbDistance,pbPower);

        tempFitTerms(useMaskArr(idx)) = tempFitTerms(useMaskArr(idx)) + delStep;
        [delReal2, delImag2] = calc_ps_cont_response(tempFitTerms,...
            uVals,vVals,freqVals,pbDistance,pbPower);

        xVals(:,idx) = [(delReal2-delReal1); (delImag2-delImag1)];
    end

end

function [realVals, imagVals] = calc_ps_cont_response(fitParams,...
    uVals,vVals,freqVals,pbDistance,pbPower)
    speedC = 29979245800;

    complexVals = complex(zeros(size(uVals)));
    %fitParams = raOff, decOff, fluxVal, sIdxVal, nu0Val
    for idx=1:size(fitParams,2)
        if isempty(pbDistance)
            pbAtten = 0;
        else
            pbAtten = interp1(pbDistance,log(pbPower),sqrt(sum(fitParams(1:2,idx).^2)).*freqVals./speedC);
        end

        complexVals = complexVals + (fitParams(3,idx).*(freqVals./fitParams(5,idx)).^fitParams(4,idx)).*...
            exp(pbAtten + ((2j*pi).*((uVals.*fitParams(1,idx))+(vVals.*fitParams(2,idx)))));
    end
    realVals = real(complexVals);
    imagVals = imag(complexVals);
end